### Explore the Dataset 

In [2]:
from IPython.display import display
import pandas as pd
# Define the path to your local file
local_file_path = "./data/energy_data_correct.csv"

# Read the local file into a pandas DataFrame
df = pd.read_csv(local_file_path)


print("Sample of Dataset")
display(df.head())

print("\n Describe Dataset")
display(df.describe())

Sample of Dataset


,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28



 Describe Dataset


,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


### Conenct to your workspace

In [1]:
import os

subscription_id = os.getenv('subscription_id')
resource_group = os.getenv('resource_group')
workspace_name = os.getenv('workspace_name')

# You can now use these variables in your Python code
print("Subscription ID:", subscription_id)
print("Resource Group:", resource_group)
print("Workspace Name:", workspace_name)


Subscription ID: b74a5367-8189-4c03-b954-e3a41b64e370
Resource Group: dem-for-env
Workspace Name: demo-play-ground


In [3]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import os

# Get the environment variables for key_vault_url
# Azure Key Vault base URL


# Connect to the AzureML workspace
subscription_id = os.getenv('subscription_id')
resource_group = os.getenv('resource_group')
workspace_name = os.getenv('workspace_name')

# Connect to the AzureML workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)


### Upload and Register the dataset into Azure


In [7]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes,InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input


ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)
# Set the version number of the data asset (for example: '1')
VERSION = "4"

#local path
path = "./data/energy_data_correct.csv"

# Define the Data asset object
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="this is an energy building dataset, I am making a test here",
    name="energy-building-for-ml",
    version=VERSION,
)

# Create the data asset in the workspace
ml_client.data.create_or_update(my_data)


Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'energy-building-for-ml', 'description': 'this is an energy building dataset, I am making a test here', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/b74a5367-8189-4c03-b954-e3a41b64e370/resourceGroups/dem-for-env/providers/Microsoft.MachineLearningServices/workspaces/demo-play-ground/data/energy-building-for-ml/versions/4', 'Resource__source_path': None, 'base_path': '/Users/saeed/Documents/GitHub/Azure_ML_Energy_Predict', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x14fd170a0>, 'serialize': <msrest.serialization.Serializer object at 0x14fd17010>, 'version': '4', 'latest_version': None, 'path': 'azureml://subscriptions/b74a5367-8189-4c03-b954-e3a41b64e370/resourcegroups/dem-for-env/workspaces/demo-play-ground/datastores/workspace

In [19]:
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential



ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)
# Name assigned to the compute cluster
cpu_compute_target = "aml-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS11_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=2,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=120,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

Creating a new cpu compute target...


### Call the training script and log the metrics

In [15]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="conda_env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)


ml_client.environments.create_or_update(env_docker_conda)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/b74a5367-8189-4c03-b954-e3a41b64e370/resourceGroups/dem-for-env/providers/Microsoft.MachineLearningServices/workspaces/demo-play-ground/environments/docker-image-plus-conda-example/versions/2', 'Resource__source_path': None, 'base_path': '/Users/saeed/Documents/GitHub/Azure_ML_Energy_Predict', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x15b80b820>, 'serialize': <msrest.serialization.Serializer object at 0x15b80b4c0>, 'version': '2', 'latest_version': None, 'conda_file': {'dependencies': ['python=3.9.13', {'pip': ['azureml-defaults~=1.34.0', 'scikit-learn==1.2.2', 'xgboost==1.7.3', 'numpy==1.23.5', 'panda

In [20]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes,InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input
from azure.ai.ml import command


# get dataset id 
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

data_asset = ml_client.data.get("energy-building-for-ml", version="4")




job = command(
    code="./src",
    command="python train-model-script.py --training_data ${{inputs.training_data}}",
    environment=env_docker_conda,
    compute="aml-cluster",
    display_name="building-energy-train",
    experiment_name="building-energy-training-with-env-conda", 
    tags={"model_type": "randomforest", "model": "Building energy dataset"},
    inputs={'training_data': Input(path=data_asset.id,
              type=AssetTypes.URI_FILE,
              mode=InputOutputModes.RO_MOUNT)}  # Reference the dataset by name
)

# Submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/silver_chain_1d2stfjs46?wsid=/subscriptions/b74a5367-8189-4c03-b954-e3a41b64e370/resourcegroups/dem-for-env/workspaces/demo-play-ground&tid=b33be5d6-5072-448f-bad3-d8b66cf09736


Retrying due to transient client side error ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


In [8]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes,InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import Input
from azure.ai.ml import command
from azureml.core import  Environment


# Connect to the AzureML workspace
subscription_id = os.getenv('subscription_id')
resource_group = os.getenv('resource_group')
workspace_name = os.getenv('workspace_name')

# You can now use these variables in your Python code
print("Subscription ID:", subscription_id)
print("Resource Group:", resource_group)
print("Workspace Name:", workspace_name)



Subscription ID: b74a5367-8189-4c03-b954-e3a41b64e370
Resource Group: dem-for-env
Workspace Name: demo-play-ground


In [ ]:

job = command(
    code="./src",
    command="python train-model-mlflow.py --training_data ${{inputs.training_data}}",
    environment="myenv",
    compute="aml-cluster",
    display_name="building-energy-train",
    experiment_name="building-energy-training", 
    tags={"model_type": "randomforest", "model": "Building energy dataset"},
    inputs={'training_data': Input(path=data_asset.id,
              type=AssetTypes.URI_FILE,
              mode=InputOutputModes.RO_MOUNT)}  # Reference the dataset by name
)

# Submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [3]:
import sklearn
import xgboost
import argparse
import numpy
import pandas

# print("Python version:", platform.python_version())
print("scikit-learn version:", sklearn.__version__)
print("xgboost version:", xgboost.__version__)
print("argparse is a standard library, its version is tied with Python's version.")
print("numpy version:", numpy.__version__)
print("pandas version:", pandas.__version__)


scikit-learn version: 1.2.2
xgboost version: 1.7.3
argparse is a standard library, its version is tied with Python's version.
numpy version: 1.23.5
pandas version: 1.5.3


In [16]:
from azureml.core import Workspace, Environment

# Create a Workspace object
workspace = Workspace(subscription_id, resource_group, workspace_name)

# Get the environment
env_name = "myenv"  # replace with your environment name
myenv = Environment.get(workspace, env_name)

print(myenv)


Environment(Name: myenv,
Version: 1)
